In [1]:
from topicsfinder import TopicsFinder
from textfilereader import TextFileReader

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/kapangyu/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kapangyu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kapangyu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
reader = TextFileReader('./sample_data/CSS_Hiring_Data_FedEmployee_Reason_OTHER_v1.xlsx')
data = reader.get_dataframe('Reason for filling position(s) with Federal Government Employee -OTHER')
data.head()

,TEXT
0,casting a wider net.
1,VRA
2,I performed both internal and external hiring.
3,haven't been able to advertise other than inte...
4,External hiring requires alot more approvals a...


In [4]:
# num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.


In [5]:
finder = TopicsFinder(data)


In [9]:
import numpy as np
import pandas as pd
import tqdm
import gensim

grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')
# Validation sets
corpus = finder.corpus
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(corpus, num_of_docs*0.75), 
               corpus]
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    # cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                    #                               k=k, a=a, b=b)
                    mod, cv = finder.fit_LDA_model(
                        random_state=100,
                        chunksize=chunksize,
                        passes=passes,
                        iterations=iterations,
                        eval_every=eval_every,
                        num_topics = k,
                        alpha = a,
                        eta = b,

                    )
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()

100%|██████████| 540/540 [3:24:21<00:00, 23.50s/it]

NameError: name 'pd' is not defined